### **Cargue de datos**


In [11]:
# Important libraries
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql.types import StructType
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import FloatType, StringType, IntegerType, DateType
from pyspark.sql.functions import udf, col, length, isnan, when, count
from pyspark.sql.functions import col, to_date
import pyspark.sql.functions as f
from pyspark.sql.types import NumericType
import os 
from datetime import datetime
from pyspark.sql import types as t
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
from pyspark.sql.functions import col

In [6]:
class MySQLConnector:
    def __init__(self, spark: SparkSession, connection_properties: dict, url: str):
        self.spark = spark
        self.properties = connection_properties
        self.url = url

    def get_dataframe(self, sql_query: str):
        """
        Execute a SQL query and return the result as a Spark DataFrame.

        Parameters:
        sql_query (str): SQL query to be executed.

        Returns:
        DataFrame: Resultant DataFrame from the executed SQL query.
        """
        df = self.spark.read.jdbc(
            url=self.url,
            table=sql_query,
            properties=self.properties
        )
        return df

In [7]:
# Utils to create spark object
def create_spark_session(path_jar_driver):
    # Configuración de la sesión de Spark
    conf = SparkConf() \
        .set('spark.driver.extraClassPath', path_jar_driver)

    spark_context = SparkContext(conf=conf)
    sql_context = SQLContext(spark_context)
    spark = sql_context.sparkSession

    return spark


In [8]:
# Given Credentials
db_user = 'Estudiante_65_202415'
db_psswd = 'Estudiante_202010409'

connection_properties = {
    "user": db_user,
    "password": db_psswd,
    "driver": "com.mysql.cj.jdbc.Driver"
}

default_string_connection = 'jdbc:mysql://157.253.236.120:8080/WWImportersTransactional'

In [9]:
# WINDOWS - Compatible with VM  
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

# LINUX users: to download the jar wget https://dev.mysql.com/get/Downloads/Connector-J/mysql-connector-java-8.0.28.tar.gz
#path_jar_driver = '/opt/mysql/lib/mysql-connector-java-8.0.28.jar'

In [4]:
spark = create_spark_session(path_jar_driver)

24/11/02 08:34:07 WARN Utils: Your hostname, willp resolves to a loopback address: 127.0.1.1; using 192.168.1.113 instead (on interface wlp9s0)
24/11/02 08:34:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/02 08:34:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/02 08:34:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/willp/anaconda3/lib/python3.11/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [25]:
conn = MySQLConnector(spark=spark, connection_properties=connection_properties, url=default_string_connection)

In [30]:
sql_conditions = 'RaSaTransaccional.FuenteCondicionesDePago_Copia_E'

In [31]:
benefits  = conn.get_dataframe(sql_conditions)
benefits.show()

+-----------------+--------------------+--------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+-----+
|IdTipoBeneficio_T|              Nombre|     UnidadDelLimite|EsEHB|EstaCubiertaPorSeguro|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|Fecha|
+-----------------+--------------------+--------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+-----+
|              565|Nutritional Couns...|                    |   No|                  Yes|                     No|                                      No|                                    Yes| 2017|
|              795|Rehabilitative Sp...|Days per Benefit ...|  Yes|                  Yes|                    Yes|                                      No|                                    Yes| 2

## **Perfilamiento de datos**

### **Entendimiento de los datos**

### **Revisión de reglas de negocio:** 

### **Análisis de calidad de datos**

#### **Completitud y Validez de los datos**

#### **Cardinalidad**

#### **Consistencia de los datos**

### **Analisis Descriptivo**

### Conclusiones

Las conclusiones se van consolidad en un solo documento word.
